In [1]:
!pip install transformers
!pip install datasets
!pip install torch


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 64.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 49.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 29.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 49.6 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [2]:
import pandas as pd

# قراءة الملف
df = pd.read_csv("merged_reuters_cnbc.csv")

# معاينة الأعمدة وعدد القيم الفارغة
print(df.columns)
print(df.isnull().sum())

# نعرض أول 3 أوصاف لنتأكد من جودة النصوص
print(df["Description"].dropna().head(3))


Index(['Headlines', 'Time', 'Description'], dtype='object')
Headlines      280
Time           280
Description    280
dtype: int64
0    TikTok has been in discussions with the UK gov...
1    Walt Disney  has become the latest company to ...
2    Former Wirecard  chief operating officer Jan M...
Name: Description, dtype: object


In [3]:
def clean_text(text):
    if pd.isna(text):
        return ""
    text = str(text)
    text = text.replace('\n', ' ').replace('\r', ' ')
    text = text.replace('&nbsp;', ' ')
    text = text.strip()
    return text

df["clean_description"] = df["Description"].apply(clean_text)
sample_texts = df["clean_description"].dropna().head(20).tolist()


In [4]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline

# تحميل النموذج والتوكنرايزر
model_name = "dslim/bert-base-NER"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForTokenClassification.from_pretrained(model_name)

# إنشاء الـ pipeline مع تفعيل تجميع الكيانات
nlp_ner = pipeline("ner", model=model, tokenizer=tokenizer, grouped_entities=True)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/59.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/829 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/433M [00:00<?, ?B/s]

Some weights of the model checkpoint at dslim/bert-base-NER were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cpu
/usr/local/lib/python3.11/dist-packages/transformers/pipelines/token_classification.py:170: UserWarning: `grouped_entities` is deprecated and will be removed in version v5.0.0, defaulted to `aggregation_strategy="AggregationStrategy.SIMPLE"` instead.
  warnings.warn(


In [ ]:
for i, text in enumerate(sample_texts):
    print(f"\n وصف الخبر رقم {i+1}:\n{text}\n")
    entities = nlp_ner(text)
    print(" الكيانات المستخرجة:")
    for ent in entities:
        print(f" - {ent['word']} --> {ent['entity_group']} ({ent['score']:.2f})")



📰 وصف الخبر رقم 1:
TikTok has been in discussions with the UK government over the past few months to locate its headquarters in London, a source familiar with the matter said, as part of a strategy to distance itself from its Chinese ownership.

📌 الكيانات المستخرجة:
 - TikTok --> ORG (0.98)
 - UK --> LOC (1.00)
 - London --> LOC (1.00)
 - Chinese --> MISC (1.00)

📰 وصف الخبر رقم 2:
Walt Disney  has become the latest company to slash its advertising spending on Facebook Inc  as the social media giant faces an ad boycott over its handling of hate speech and controversial content, the Wall Street Journal reported on Saturday, citing people familiar with the situation.

📌 الكيانات المستخرجة:
 - Walt Disney --> ORG (0.73)
 - Facebook Inc --> ORG (1.00)
 - Wall Street Journal --> ORG (1.00)

📰 وصف الخبر رقم 3:
Former Wirecard  chief operating officer Jan Marsalek travelled to Minsk soon after he was suspended and may still be in Belarus or Russia, a German magazine reported on Saturday.

📌